<a href="https://colab.research.google.com/github/Bora-Ulu/Inflation-Technique/blob/main/Inflation_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import *
from itertools import *
import time
from scipy.sparse import coo_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix
from cvxopt import matrix, solvers, sparse, spmatrix
from numba import njit

@njit
def PrepForIntToArray32(ar):# Takes a huge column vector as input
                     # Returns a matrix where the rows are the base-4 binary expansions of each element of the input array
    #return transpose(atleast_2d(ravel(ar.astype(uint32)))).view(uint8)
    return reshape(ar.astype(uint32).view(uint8),(-1,4))


def IntToArray32(ar):
    return unpackbits(PrepForIntToArray32(ar),axis=1, bitorder='little').view(bool_)


def ArrayToInt32(bitarray):
    return packbits(bitarray, bitorder='little').view(uint32)

@njit
def ExtractBitsFromBitArray(bitarray,bitstoextract): #bitstoextract must be numpy array
    tempbitarray=zeros_like(bitarray)
    tempbitarray[:,:len(bitstoextract)]=bitarray[:,subtract(amax(bitstoextract),bitstoextract)]
    return tempbitarray

def ExtractBitsFromInts(intarray,bitstoextract): #bitstoextract need not be a numpy array

    return ArrayToInt32(ExtractBitsFromBitArray(IntToArray32(intarray),array(bitstoextract,uint16)))

@njit
def PositionIndex(arraywithduplicates):
    arraycopy=zeros_like(arraywithduplicates)
    u=unique(arraywithduplicates)
    arraycopy[u]=arange(len(u))
    return arraycopy[arraywithduplicates]

In [2]:
def SwappingOps(card, num_var, anc_con):
    c=int(card/2)
    a=array([list(arange(2)+(x*c)) for x in range(num_var)])
    
    for i in range(len(anc_con)):
        
        C=list(combinations(list(anc_con[i]),2))
        
        for j in range(len(C)):
                
            V=vstack((C[j][0],C[j][1]))
                
            V=a[V]
            S1=ravel(V[0])
            S2=ravel(V[1])
                
            Swap=arange(32)          
            Swap[(num_var*c-1)-hstack((S1,S2))]=Swap[(num_var*c-1)-hstack((S2,S1))]
                
            if i==0 and j==0:
                    
                Swaps=Swap
                    
            else:
                    
                Swaps=vstack((Swaps,Swap))
                
    return Swaps

card=4
num_var=12
X=array([[0,1,8,10],[2,3,9,11]])
Y=array([[0,2,4,5],[1,3,6,7]])
Z=array([[4,6,8,9],[5,7,10,11]])
ai=[0,1,6,7,8,9,14,15,16,17,22,23]
anc_con=[X,Y,Z]

N=SwappingOps(card, num_var, anc_con)

In [3]:
def A(card, num_var, anc_con):
    
    ColumnIntegers=arange(0,card**num_var,1,uint32)
    
    ColumnIntegersAsBits=IntToArray32(ColumnIntegers)
    
    S=SwappingOps(card, num_var, anc_con)
    
    multiplier=hstack((2**arange(num_var*2),zeros((32-2*num_var),uint32)))
    
    SymMultiplier=multiplier
    
    a=range(len(S))
    
    for i in range(1,len(S)+1):
        
        C=list(combinations(a,i))
        
        for k in C:
            
            D=S[k[0]]
            j=0
            while j+1 < i:
                
                D=D[S[k[j+1]]]
                j=j+1
            
            SymMultiplier=vstack((SymMultiplier,multiplier[D]))
    A=dot(SymMultiplier[:,:(2*num_var)],ColumnIntegersAsBits.T[:(2*num_var),:])
    print(A.shape)
    A=A[:,unique(amin(A,axis=0))]
    print(A.shape)
    return A

L=A(card, num_var, anc_con)

(8, 16777216)
(8, 2123776)


In [4]:
def MergeMonomials(bvector,encoding):
    return ravel(coo_matrix((bvector, (zeros(len(bvector),uint8), encoding)),(1, amax(encoding)+1)).toarray())

In [36]:
inflationorder=2;
ai_naturalorder=[0,1,8,9,16,17,6,7,14,15,22,23]
"""ai_sym=ravel(reshape(ai_naturalorder,(2,-1)).T)"""
ai_sym=reshape(array(list(permutations(reshape(ai_naturalorder,(inflationorder,-1)).tolist()))),(-1,len(ai_naturalorder)))
RowIntegers=RowIntegers=arange(0,card**(int(len(ai)/2)),1,uint32)
for i in range(1,len(ai_sym)):
    RowIntegers=vstack((RowIntegers,ExtractBitsFromInts(RowIntegers,ai_sym[i])))
amin(RowIntegers,axis=0)

array([   0,    1,    2, ..., 3116, 3100, 3132], dtype=uint32)

In [47]:
def EncodeA_FindB(card, num_var, anc_con, ai_naturalorder, Data, inflationorder):
    
    RowIntegers=arange(0,card**(int(len(ai)/2)),1,uint32)
    
    ColumnIntegers=arange(0,card**num_var,1,uint32)
    
    ColumnIntegersAsBits=IntToArray32(ColumnIntegers)
    """
    ai_sym=arange(len(ai))
    
    i=0
    
    while i<=len(ai_sym)-4:
        
        ai_sym[[i,i+1,i+2,i+3]]=ai_sym[[i+2,i+3,i,i+1]]
        
        i=i+4
    
    ai_sym=list(ai_sym)
    """
    ai_sym=reshape(array(list(permutations(reshape(ai_naturalorder,(inflationorder,-1)).tolist()))),(-1,len(ai_naturalorder)))
    for i in range(1,len(ai_sym)):
        RowIntegers=vstack((RowIntegers,ExtractBitsFromInts(RowIntegers,ai_sym[i])))
    
    EncodingMonomialToRow=PositionIndex(amin(RowIntegers,axis=0))
    
    
    EncodingColumnToMonomial=ArrayToInt32(ExtractBitsFromBitArray(ColumnIntegersAsBits,array(ai_naturalorder,uint16)))
    
    EncodedA=EncodingMonomialToRow[EncodingColumnToMonomial][A(card, num_var, anc_con)]
    
    preb=array(Data)
    b=preb
    for i in range(1,inflationorder):
        b=kron(preb,b)

    b=MergeMonomials(b,EncodingMonomialToRow)

    return EncodedA,b

In [7]:
def FormCVXOPTArrayFromOnesPositions(OnesPositions):
    columncount=OnesPositions.shape[-1]
    columnspec=ravel(broadcast_to(arange(columncount), (len(OnesPositions), columncount)))
    return spmatrix(ones(OnesPositions.size), ravel(OnesPositions).tolist(), columnspec.tolist(),(amax(OnesPositions)+1, columncount))

In [48]:
Data=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
inflationorder=2;
ai_naturalorder=[0,1,8,9,16,17,6,7,14,15,22,23]
start = time.time()
EncodedA,b=EncodeA_FindB(card, num_var, anc_con, ai_naturalorder,Data,inflationorder)
MCVXOPT=FormCVXOPTArrayFromOnesPositions(EncodedA).T #This far the code is generalized
print('It took', time.time()-start, 'seconds.')
print(MCVXOPT.size)

(8, 16777216)
(8, 2123776)
(4096,)
It took 25.823704957962036 seconds.
(2123776, 1624)


In [9]:
rowcount=MCVXOPT.size[0];
colcount=MCVXOPT.size[1];
CVXOPTb=matrix(atleast_2d(b).T)
CVXOPTh=matrix(zeros((rowcount,1)))
CVXOPTA=matrix(ones((1,colcount)))
solvers.lp(CVXOPTb,-MCVXOPT,CVXOPTh,CVXOPTA,matrix(ones((1,1))),solver='mosek')

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.67            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 10.66   
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization 

{'status': 'optimal',
 'x': <2080x1 matrix, tc='d'>,
 's': <2123776x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 'z': <2123776x1 matrix, tc='d'>,
 'primal objective': -1.9506798565794375e-05,
 'dual objective': -1.9506798604887057e-05,
 'gap': 1.2090503497545257e-13,
 'relative gap': 6.198097271966624e-09,
 'primal infeasibility': 2.3159252293680765e-11,
 'dual infeasibility': 0.0005902077338705597,
 'residual as primal infeasibility certificate': None,
 'residual as dual infeasibility certificate': None,
 'primal slack': -3.6155888314781886e-12,
 'dual slack': -1.8737150662485965e-06}

In [10]:
#A1 A2 A3 A4 B1 B2 B3 B4 C1 C2 C3 C4
#0. 1. 2. 3. 4. 5. 6. 7. 8. 9. 10 11

#A1. A2. A3. A4. B1. B2.   B3.   B4.   C1.   C2.   C3.   C4
#0,1 2,3 4,5 6,7 8,9 10,11 12,13 14,15 16,17 18,19 20,21 22,23

#A1 A4 B1 B4 C1 C4
#0. 1. 2. 3. 4. 5